## Load data

In [45]:
import pandas
training = pandas.read_csv('training.tsv', delimiter='\t')
testing = pandas.read_csv('test.tsv', delimiter='\t')
training.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,Class
0,1,1799,390,26,6,9,7,4,27,7,4,27,4
1,1,1802,390,18,5,98,8,3,32,8,3,32,4
2,4,560,2667,21,5,13,6,3,23,6,3,23,4
3,1,988,390,10,3,120,7,2,27,7,2,27,4
4,4,2129,390,23,6,17,6,2,26,6,2,26,4


## Feature selection
https://machinelearningmastery.com/feature-selection-machine-learning-python/


In [27]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
import pandas
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
classIndex = len(training.columns)-1
lastColIndex = classIndex-1
array = training.values
X = array[:,0:-2]
Y = array[:,-1]
# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)
# summarize scores
columns = list(training.columns)
numpy.set_printoptions(precision=3)
scoredFeatures = [{"score":x,"column":columns[i]} for i,x in enumerate(fit.scores_)]
sortedScoredFeatures = sorted(scoredFeatures, key=lambda x: x['score'], reverse=True) 
print(pandas.DataFrame.from_dict(sortedScoredFeatures))


   column         score
0      A1  6.922209e+07
1      A2  1.176584e+07
2      A3  9.947493e+06
3      A4  3.012508e+05
4      A6  1.659314e+05
5      A9  7.330293e+04
6      A5  6.170595e+04
7      A7  1.552336e+04
8     A10  1.549642e+04
9      A8  2.239109e+02
10    A11  1.973365e+02


## Classification
### auto_ml
http://auto-ml.readthedocs.io/en/latest

In [40]:
# http://auto-ml.readthedocs.io/en/latest/
from auto_ml import Predictor

# If you pass in any categorical data as a number, tell us here and we'll take care of it.
col_desc_dictionary = {"Class": "output"}

# Can pass in type_of_estimator='regressor' as well
ml_predictor = Predictor(type_of_estimator='classifier', column_descriptions=col_desc_dictionary)

# Wait for the machine to learn all the complex and beautiful patterns in your data...
ml_predictor.train(training, ml_for_analytics=True)

# And this time, in your shell, it will print out the results for what it found was useful in making predictions!

# Where new_data is a single dictionary, or a DataFrame
ml_predictor.predict(test)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at https://github.com/ClimbsRocks/auto_ml
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'learning_rate': 0.1, 'warm_start': True, 'presort': False}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'learning_rate': 0.1, 'warm_start': True, 'presort': False}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict Cla

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

### TPOT
https://rhiever.github.io/tpot


In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train = training[training.columns[:-1]]
X_test = testing[testing.columns[:-1]]
y_train = training[training.columns[-1]]
y_test = testing[testing.columns[-1]]

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
# pipeline_optimizer.export('tpot_exported_pipeline.py')

Optimization Progress:   0%|                                                             | 0/120 [00:00<?, ?pipeline/s]

In [43]:
training[training.columns[:-2]]

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11
0,1,1799,390,26,6,9,7,4,27,7,4
1,1,1802,390,18,5,98,8,3,32,8,3
2,4,560,2667,21,5,13,6,3,23,6,3
3,1,988,390,10,3,120,7,2,27,7,2
4,4,2129,390,23,6,17,6,2,26,6,2
5,405,1634,6,17,4,21,5,5,20,5,5
6,1,1100,2057,11,3,4,3,6,11,3,6
7,4,2667,3115,17,4,27,5,7,20,5,7
8,1,390,1799,27,7,78,9,2,38,9,2
9,405,1,90,20,5,73,7,5,30,7,5
